In [12]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## 1. Грузим данные и тренируем модельку

In [25]:
# Load the diabetes dataset
X, y = datasets.load_breast_cancer(return_X_y=True)

# # Use only one feature
# X = X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create linear regression object
regr = linear_model.LogisticRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

/Users/stanislav.nosulenko/opt/anaconda3/envs/analytics/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 2. Оцениваем ДИ ROC AUC

In [35]:
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

def smooth_bootstrap_auc(data, target, n_iterations=1000, alpha=0.05):
    """
    Calculates the bootstrapped confidence interval for the ROC AUC metric.

    :param data: array-like, shape = [n_samples, n_features]
                 The input samples.
    :param target: array-like, shape = [n_samples]
                   The target values (binary classification).
    :param n_iterations: int, default=1000
                         The number of bootstrap samples to use.
    :param alpha: float, default=0.05
                  The significance level (for a 95% confidence interval, alpha=0.05).

    :return: tuple(float, float)
             The lower and upper bound of the confidence interval.
    """
    bootstrapped_scores = []

    for _ in range(n_iterations):
        # Bootstrap resample
        indices = resample(np.arange(len(data)), replace=True)
        if len(np.unique(target[indices])) < 2:
            # If sample doesn't have both classes, skip this iteration
            continue

        score = roc_auc_score(target[indices], data[indices])
        bootstrapped_scores.append(score)

    # Calculate the percentiles for the confidence interval
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    # 100*(1-alpha) confidence interval
    lower = np.percentile(sorted_scores, 100*alpha/2)
    upper = np.percentile(sorted_scores, 100*(1-alpha/2))

    return lower, upper

In [36]:
AUC_l, AUC_u = smooth_bootstrap_auc(y_pred, y_test)
AUC = roc_auc_score(y_test, y_pred)
print('AUC:', AUC)
print('95% CI:', AUC_l, AUC_u )

AUC: 0.9107142857142856
95% CI: 0.8525595238095238 0.9605263157894737
